<a href="https://colab.research.google.com/github/shake/colab-Llama-2-ipynb/blob/main/Quantize_Llama-2-7b-chat_with_AutoGPTQ_and_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load required libraries

Let us first load the required libraries that are 🤗 transformers, optimum and auto-gptq library.

In [1]:
%%capture
!pip install transformers optimum accelerate peft trl auto-gptq bitsandbytes datasets

In [2]:
# import 密钥，token
%%capture
from google.colab import userdata
hf_token = userdata.get('huggingface')
!git config --global credential.helper store
!huggingface-cli login --token $hf_token --add-to-git-credential


## 🚀 **Two Ways to Use a Quantized Model**:

1. **From Scratch**: Quantize a new language model.

2. **Pre-Quantized Models**: Load an already quantized model from 🤗 Hub.

🔬 **GPTQ Needs Calibration**: Requires inference on the quantized model to calibrate quantized weights.

📖 **Auto-GPTQ Process**:

- **Dataset Requirement**: For quantization, provide a dataset.

- **Choices**: Use default datasets like 'wikitext2', 'c4', etc., or your own list of strings.

🧪 **How It Works**: Detailed algorithm in the [original paper](https://arxiv.org/pdf/2210.17323.pdf).

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
import torch

In [ ]:
model_id = "chenshake/Llama-2-7b-chat-hf"

In [ ]:
quantization_config = GPTQConfig(
     bits=4,
     group_size=128, # The group size to use for quantization; default value
     dataset="wikitext2", # The dataset used for calibration.
     desc_act=False,# Whether to quantize columns in order of decreasing activation size. Setting it to False can significantly speed up inference but the perplexity may become slightly worse.



In [ ]:
quantization_config = GPTQConfig(
     bits=4,
     group_size=128, # The group size to use for quantization; default value
     dataset="wikitext2", # The dataset used for calibration.
     desc_act=False,# Whether to quantize columns in order of decreasing activation size. Setting it to False can significantly speed up inference but the perplexity may become slightly worse.

)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          trust_remote_code = True)

In [ ]:
quant_model = AutoModelForCausalLM.from_pretrained(model_id,
                                                   trust_remote_code = True,
                                                   quantization_config=quantization_config,
                                                   device_map='auto'
                                                   )

In [ ]:
quant_model.push_to_hub("Llama-2-7b-chat-hf-gptq-4bit-wikitext")

tokenizer.push_to_hub("Llama-2-7b-chat-hf-gptq-4bit-wikitext")

# backup

### This notebook was created by [Harpreet Sahota](https://twitter.com/DataScienceHarp), DevRel Manager at [Deci AI](https://deci.ai/). Keep in touch by joining me in our community, [Deep Learning Daily](https://www.deeplearningdaily.community/)

GPTQ is the Marie Kondo for Language Models - it helps tidy up LLMs, making them more efficient and faster without losing their smarts.

What is GPTQ?

It's a post-training quantization technique for large language models, that uses second order information about the weight matrices for better quantization.

Second-order information basically means second order derivatives, such as the Hessian matrix, to perform weight quantization.

These second-order derivatives provide information about the curvature of the loss landscape, and GPTQ to capture more nuanced characteristics of the model parameters and improve the quantization process

The end result is that GPTQ effectively reduces the computational and memory requirements of large language models without significant loss of accuracy. Making them more efficient to deploy by reducing their size and computational requirements.

# 🔍 **GPTQ Algorithm**

1. **Arbitrary Order Insight**

    - Traditional quantization methods quantize weights in a specific order to minimize error.

    - [GPTQ: Accurate Post-Training Quantization for Generative Pre-trained Transformers](https://arxiv.org/abs/2210.17323) is an algorithm to compress any transformer-based language model into few bits with minimal performance degradation.

   - GPTQ's cool insight: Just mix it up, order doesn't matter much, especially with big, complex layers.
   
   - Traditional methods prioritize quantizing weights in a specific sequence to minimize quantization error.
     - However, GPTQ discovered that for large, heavily-parametrized layers, quantizing weights in arbitrary order is nearly as effective.
   
   - This approach maintains similar error levels while simplifying the process.

2. **Quantizing All Rows in Same Order**

   - Instead of treating each row like it's special, GPTQ quantizes every row the same way.

    - GPTQ quantizes all weights in the same order across rows, a departure from row-specific order.

   - This method significantly reduces runtime, especially for larger models.

   - Utilizes inverse Hessian information for quantization.

3. **Lazy Batch-Updates for Efficiency**

   - GPTQ groups columns, updates them together, and BAM! Much faster, especially on GPUs.

   - GPTQ batches updates to improve GPU utilization, applying the algorithm to groups of columns at a time.

   - This batch update approach doesn't reduce compute but alleviates memory bandwidth bottlenecks, offering substantial speedups.

4. **Cholesky Reformulation for Numerical Stability**

   - Addresses numerical inaccuracies, a significant issue at larger scales.

   - Uses a [Cholesky decomposition approach](https://www.seas.ucla.edu/~vandenbe/133A/lectures/chol.pdf) to precompute necessary information, ensuring robustness on huge models.

   - Mild dampening is applied to diagonal elements of Hessian matrix to avoid numerical issues.

### The end result?

🚀 Reduces model size significantly, enabling use on less powerful hardware, like fewer GPUs.

🤖 Ideal for giant models with billions of parameters.

⏱ Can quantize up to 175 billion parameters in about 4 hours.

🔍 Reduces bitwidth to 3-4 bits per weight with little accuracy loss.

💡 Makes using and accessing large language models more practical and accessible.


##🔧 **Quantizing and Precision**:

- **Precision Level**: Opting for 4-bit precision.

- **Dataset Choice**: Using the `"wikitext2"` dataset for this example.

- **Supported Precisions**: Supported numbers are (2, 3, 4, 8).

⏳ **Patience is Key**: The process can take a long time.

## 📚 **Calibration Datasets in GPTQ**

To ensure that the model is quantized with minimal difference between the expected output of the original model and the quantized model, it's important to use a representative set of data that the model is expected to encounter during actual usage. Adequate amount of data is also crucial.

- **Critical Role**: Calibration datasets adjust quantized weights to match original model performance.

- **Fine-Tuning Necessity**: Inference on calibration data helps fine-tune the quantized model.

- **Quality Counts**: The effectiveness of quantization heavily depends on the dataset's representativeness.

- **For Custom Datasets**: Check the next section for quantizing with a custom dataset.

🤔 **Dataset Calibration Intuition**

- **Mimicking Real-World Data**: Calibration ensures the quantized model behaves like it would on actual data.

- **Adjusting Quantized Weights**: Helps fine-tune the model's parameters to closely resemble the original, full-precision model.

- **Balancing Performance and Size**: Aims to maintain performance while benefiting from reduced model size due to quantization.



🔍 **Checking Quantization Success**:

- **Key Attributes**: Look for `qweight` and `qzeros` in the linear layers.

- **Data Type**: These attributes should be in `torch.int32` dtype.

👀 **Validation Step**: This check ensures the model is correctly quantized.

🔧 **Linear Layers & Quantization**

- **Linear to QuantLinear**: Linear layers are transformed to `QuantLinear` modules using auto-gptq library.

- **Exllama Kernel Use**: Whether to use exllama backend. Only works with bits = 4. To use a model on a configuration with limited VRAM split across multiple devices with device_map, disable exllama by setting disable_exllama to True. True means exllama support is set to False, which is enabled by default. (`disable_exllama = False`).

- **Bit Specificity**: This kernel is compatible only with 4-bit models.

In [5]:
quant_model.model.layers[0].self_attn

LlamaAttention(
  (rotary_emb): LlamaRotaryEmbedding()
  (k_proj): QuantLinear()
  (o_proj): QuantLinear()
  (q_proj): QuantLinear()
  (v_proj): QuantLinear()
)

In [6]:
quant_model.model.layers[0].self_attn.q_proj.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict([('qweight',
               tensor([[-2002352041, -2004318071, -2004318072,  ..., -1751537512,
                        -1987733349, -1735813002],
                       [-2006349400, -1986492536, -2022143864,  ...,  1946716311,
                        -1149724502,  1988597867],
                       [ 2071492471, -2121754472, -1886877576,  ..., -1781037193,
                         2020121496, -2037807225],
                       ...,
                       [-1989183607,  2040174184,  2003065448,  ..., -2029234553,
                        -1913538456,  1922734263],
                       [-1752659848, -1984456569, -2005304471,  ...,  1940510857,
                        -1938212505, -2005301817],
                       [ 2023266488, -1770497898, -1702320021,  ..., -1785104504,
                         1985403463,  2058831801]], device='cuda:0', dtype=torch.int32)),
              ('qzeros',
               tensor(

In [7]:
quant_dict = quant_model.config.quantization_config.to_dict()

In [8]:
quant_dict.keys()

dict_keys(['quant_method', 'bits', 'tokenizer', 'dataset', 'group_size', 'damp_percent', 'desc_act', 'sym', 'true_sequential', 'use_cuda_fp16', 'model_seqlen', 'block_name_to_quantize', 'module_name_preceding_first_block', 'batch_size', 'pad_token_id', 'use_exllama', 'max_input_length', 'exllama_config', 'cache_block_outputs', 'modules_in_block_to_quantize'])

🚀 **Running Inference on Quantized Model**

- **Same Old, New Twist**: Use the familiar transformers API for inference.

- **Seamless Integration**: The quantized model works just like the regular transformers models.

In [ ]:
decilm_4_bit_model = AutoModelForCausalLM.from_pretrained(
    'harpreetsahota/DeciLM-7B-Instruct-gptq-4bit-slim-orca',
    trust_remote_code=True,
    use_cache=False,
    device_map="auto",
    low_cpu_mem_usage=True
    )

tokenizer = AutoTokenizer.from_pretrained(
    'harpreetsahota/DeciLM-7B-Instruct-gptq-4bit-slim-orca',
    trust_remote_code = True
    )

In [ ]:
from transformers import pipeline, TextStreamer

system_prompt = "You are an AI assistant that follows instruction extremely well. Help as much as you can."

user_prompt = """You are the rapper Eazy-E. Write a rap about how you just ate the most amazing pancakes ever.\
Use lots of details to describe your experience.
"""

prompt = ([
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
    ])

streamer = TextStreamer(tokenizer)

pipe = pipeline("conversational",
                model=decilm_4_bit_model,
                tokenizer=tokenizer,
                temperature=0.1,
                max_length = 1024,
                streamer=streamer
                )

pipe(prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### System:
You are an AI assistant that follows instruction extremely well. Help as much as you can.
### User:
You are the rapper Eazy-E. Write a rap about how you just ate the most amazing pancakes ever.Use lots of details to describe your experience. 

### Assistant:
Yo, listen up, I got a story to tell
'Bout the most amazing pancakes I ever did eat
I was hungry, so I went to the kitchen
And whipped up a batch, just for me

I mixed the batter, just like I do
And poured it into the pan, nice and smooth
The sizzle and the smell, it was heavenly
I knew I was in for a treat, oh so tasty

The pancakes were golden, just like the sun
And the syrup was sweet, like a summer fun
I took a bite, and my taste buds danced
I was in pancake heaven, no need to expand

The fluffy texture, it was so soft
And the flavor, it was like a gift
I savored every bite, with a smile on my face
I knew I had to share this with my friends, no need to waste

So I called them up, and they came right away
We sat down

Conversation id: 01fde20e-5f1c-47a6-91bb-4fa6b0fd2f3b
system: You are an AI assistant that follows instruction extremely well. Help as much as you can.
user: You are the rapper Eazy-E. Write a rap about how you just ate the most amazing pancakes ever.Use lots of details to describe your experience. 

assistant: Yo, listen up, I got a story to tell
'Bout the most amazing pancakes I ever did eat
I was hungry, so I went to the kitchen
And whipped up a batch, just for me

I mixed the batter, just like I do
And poured it into the pan, nice and smooth
The sizzle and the smell, it was heavenly
I knew I was in for a treat, oh so tasty

The pancakes were golden, just like the sun
And the syrup was sweet, like a summer fun
I took a bite, and my taste buds danced
I was in pancake heaven, no need to expand

The fluffy texture, it was so soft
And the flavor, it was like a gift
I savored every bite, with a smile on my face
I knew I had to share this with my friends, no need to waste

So I called the

# Quantize to 2-bits

In [ ]:
model_id = "Deci/DeciLM-7B-instruct"

quantization_config = GPTQConfig(
    bits=2,
    group_size=128,
    use_cuda_fp16=True,
    desc_act=False,
    dataset=skinny_orca_texts,
    model_seqlen=8192
)

quant_model = AutoModelForCausalLM.from_pretrained(model_id,
                                                   quantization_config=quantization_config,
                                                   trust_remote_code=True,
                                                   use_cache=False,
                                                   torch_dtype="auto",
                                                   low_cpu_mem_usage=True
                                                   )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Quantizing model.layers blocks :   0%|          | 0/32 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
quant_model.push_to_hub("harpreetsahota/DeciLM-7B-Instruct-gptq-2bit-slim-orca")

tokenizer.push_to_hub("harpreetsahota/DeciLM-7B-Instruct-gptq-2bit-slim-orca")

model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/harpreetsahota/DeciLM-7B-Instruct-gptq-2bit-slim-orca/commit/006229e7dcaeb0ef4518d0ae759ae7a86341acd7', commit_message='Upload tokenizer', commit_description='', oid='006229e7dcaeb0ef4518d0ae759ae7a86341acd7', pr_url=None, pr_revision=None, pr_num=None)